In [1]:
import numpy as np
from collections import ChainMap
from itertools import permutations

In [2]:
# lists containing individual ranked lists (i.e., base sequences)

ADNI = ['ABETA', 'LDEL', 'MMSE', 'LIMM','PTAU','TAU', 'ENTOR', 'HIPPO', 'VENT','MIDTEMP','FUSIF']
AIBL = ['LIMM', 'LDEL', 'MMSE','FIGC']
JADNI = ['ABETA','PTAU','LIMM','MMSE','VENT','ENTOR','LDEL', 'TAU','HIPPO','FUSIF','MIDTEMP']
ANM = ['MMSE', 'CSFVOL', 'HIPPO', 'ENTOR', 'MIDTEMP', 'FUSIF', 'VENT']
WMHAD = ['MMSE', 'FUSIF','VENT', 'ENTOR', 'CSFVOL', 'HIPPO', 'MIDTEMP']
ARWIBO = ['LDEL', 'VENT', 'MMSE', 'FUSIF','LIMM', 'ENTOR', 'HIPPO', 'MIDTEMP', 'CSFVOL','FIGC']
EMIF = ['MMSE', 'ABETA', 'TAU', 'PTAU', 'LDEL', 'LIMM','HIPPO']
OASIS = ['MMSE', 'CSFVOL', 'MIDTEMP', 'HIPPO', 'ENTOR', 'FUSIF', 'VENT']
EDSD = ['MMSE', 'HIPPO', 'VENT','ENTOR', 'FIGC', 'MIDTEMP','FUSIF']
NACC = ['ABETA', 'LIMM', 'MMSE', 'TAU','PTAU','ENTOR', 'MIDTEMP', 'FUSIF', 'HIPPO', 'CSFVOL', 'VENT']

all_cohorts = [ADNI, AIBL, ANM, WMHAD, ARWIBO, EMIF, OASIS, EDSD, NACC, JADNI]

# list containing the underlying event space (ie. all possible events)
space = ['ABETA', 'LDEL', 'MMSE', 'LIMM', 'ENTOR', 'HIPPO', 'VENT','FUSIF','MIDTEMP', 'FIGC', 'CSFVOL', 'TAU', 'PTAU']


In [3]:
k = 8 # select length of starting sequence

In [4]:
def Spearman_Footrule(list_1,list_2):
    """Distance function"""
    
    dist = 0
    
    for feat in list_1:
        
        list_1_index = list_1.index(feat)
        list_2_index = list_2.index(feat)
        
        dist += abs(list_1_index - list_2_index)
    
    return dist

In [5]:
def pattern_sequence_consistency(pattern, all_cohorts):
    """Calculate distance between the potential meta-sequence / starting sequence (ie., pattern) 
    and individual event sequence"""
    
    pattern_copy = pattern.copy()
    distance_Spearman_Footrule = []
    patten_dist_Spearman_Footrule = {}

    # Handeling the partially overlapping ranked lists
    for seq in all_cohorts:
        
        pattern_common = [x for x in pattern_copy if x in seq]
        seq_common = [y for y in seq if y in pattern_copy]

        if len(seq_common) == 0:
            continue
        
        # Calculating distance between the pattern and individual event sequence
        dists_Spearman_Footrule = (Spearman_Footrule(pattern_common, seq_common) / len(seq_common))
        distance_Spearman_Footrule.append(dists_Spearman_Footrule)
    
    # Calculating the overall distance between the pattern and all cohorts
    dist_Spearman_Footrule = np.mean(distance_Spearman_Footrule)
    
    return dist_Spearman_Footrule


In [6]:
def add_remaining_feats(not_common, pattern, all_cohorts):
    """Add the remaining variables to the starting sequence (list1) in recursive manner.
    This function depends on a global dictionary called 'meta_seqs' that stores all generated
    sequences and their distances. It will later be used to select the best sequence."""
    list1 = pattern[:]
    new_not_common = not_common[:]
    
    # terminate once all variables are added
    if new_not_common == []:
        meta_seqs[tuple(list1)] = pattern_sequence_consistency(list1, all_cohorts)
        return None
  
    feat = new_not_common[0]
    dist_dict = {}

    # Iterate over all possible postions that the new variable can have in initial proposed pattern
    for i in range(len(list1) +1): # the + 1 is needed to enable "inserting" at the end

        list1.insert(i, feat)
        
        # store altered sequence and according distance
        tup = tuple()
        tup += (i, feat)
        dist_dict[tup] = pattern_sequence_consistency(list1, all_cohorts)
        
        list1.remove(feat) # remove to try next position
    
    # find all possible insetions leading to minimum distance sequences
    min_dist = min(dist_dict.values())
    possible_insertions = [insertion for insertion in dist_dict.keys() if dist_dict[insertion] == min_dist]
    new_not_common.remove(feat) # remove added variable
    
    # extend each possible option
    for possible_insertion in possible_insertions:
        
        new_list1 = list1[:] # create a copy to prevent inserting into the same seq
        new_list1.insert(*possible_insertion)
        # explore the next steps for all possible insertions
        add_remaining_feats(new_not_common, new_list1, all_cohorts)


In [ ]:
# Ste

In [7]:
#Step 1: Calculating distance between all possible start sequences (ie., patterns) and base sequence

patterns_scores_Spearman_Footrule = {}

# test all possible starting sequences using k variables
for patterns in permutations(space, k):   
    
    patterns = list(patterns)
    
    dist_Spearman_Footrule = pattern_sequence_consistency(patterns, all_cohorts)
    patterns_scores_Spearman_Footrule[tuple(patterns)] = dist_Spearman_Footrule

min_score_Spearman_Footrule = min(patterns_scores_Spearman_Footrule.values())

# Extracting the meta-sequence with the minimum distance to all event sequences
for key, value in patterns_scores_Spearman_Footrule.items():
    if value == min_score_Spearman_Footrule:
        seq = list(key)

In [8]:
# Step 2:
# Add remaining variables to initial proposed pattern coming from  
# permutation of variables in space list having the minimum distance with all event sequences
not_common = [x for x in space if x not in seq]

meta_seqs = dict() #"this dict is a global variable that will be altered in the function below"
add_remaining_feats(not_common, seq, all_cohorts)

# print meta-seqs with lowest distance
min_dist = min(meta_seqs.values())
min_meta_seqs = [key for key in meta_seqs.keys() if meta_seqs[key] == min_dist]

for meta_seq in min_meta_seqs:
    print(meta_seq)

('ABETA', 'LDEL', 'LIMM', 'MMSE', 'PTAU', 'TAU', 'CSFVOL', 'HIPPO', 'ENTOR', 'MIDTEMP', 'FUSIF', 'VENT', 'FIGC')
